<h1>Import dependencies</h1>

In [1]:
import ee
import folium
import geehydro
import geemap


C:\Users\srini\AppData\Local\Programs\Python\Python310\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [2]:
ee.Initialize(project='ml-meets-the-sun')

In [4]:
Map = geemap.Map(center = (15.3173,75.7139), zoom =5)
Map.add_basemap('HYBRID')

In [5]:
Map

Map(center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topri…

In [8]:
karnataka = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Karnataka')
TamilNadu = ee.FeatureCollection('projects/ml-meets-the-sun/assets/TamilNadu')
Rajasthan = ee.FeatureCollection('projects/ml-meets-the-sun/assets/Rajasthan')

In [26]:
karnataka

In [9]:
def map_display(state,name):
    vis_params = {
    'color': 'red'
    }
    Map.addLayer(state, vis_params, name, True, 0.5)
    return Map
Map

Map(bottom=4043.0, center=[15.3173, 75.7139], controls=(WidgetControl(options=['position', 'transparent_bg'], …

In [10]:
states = {
    'Karnataka': karnataka,
    'Tamil Nadu': TamilNadu,
    'Rajasthan': Rajasthan
}
for name,state in states.items():
    map_display(state,name)

In [11]:
era5_2mt_temp = (
    ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .select('temperature_2m')
    .filter(ee.Filter.date('2020-01-01', '2025-07-31'))
)
display(era5_2mt_temp)

In [35]:
clipped= era5_temp_mean
vis_temp ={
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
    }
Map.addLayer(clipped, vis_temp, 'cname2', True, 0.5)

In [34]:
era5_2mt_temp= era5_2mt_temp.mean().setDefaultProjection(
    era5_2mt_temp.first().projection()
)



In [36]:
era5_temp_mean.projection()

In [ ]:
setDefaultProjection(
    era5_2mt_temp.first().projection()
)


In [ ]:
result_clip = {}

def mean_temp_clip(state, name):
    cname = name + "_temp_mean"
    clipped = era5_temp_mean.clip(state)
    result_clip[cname] = clipped

In [14]:
for name,state in states.items():
    mean_temp_clip(state, name)

In [15]:
result_clip

{'Karnataka_temp_mean': <ee.image.Image at 0x206bcdd28f0>,
 'Tamil Nadu_temp_mean': <ee.image.Image at 0x206bce7bd00>,
 'Rajasthan_temp_mean': <ee.image.Image at 0x206bce7bd90>}

In [27]:
def display_mean_temp(cname, clipped):
    
    vis_temp ={
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
    }
    Map.addLayer(clipped, vis_temp, cname, True, 0.5)
    return Map

In [28]:
for cname,clipped in result_clip.items():
    display_mean_temp(cname, clipped)

In [21]:
result_clip['Karnataka_rad_mean'].projection()

In [20]:
region = karnataka.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Karnataka_temp_mean'],
    description='Karnataka_temp_mean',
    assetId='projects/ml-meets-the-sun/assets/KA_mean_temp',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [22]:
region = Rajasthan.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Rajasthan_temp_mean'],
    description='Rajasthan_mean_temp',
    assetId='projects/ml-meets-the-sun/assets/RA_mean_temp',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()


In [23]:
region = TamilNadu.geometry()

task =ee.batch.Export.image.toAsset(
    image=result_clip['Tamil Nadu_temp_mean'],
    description='TamilNadu_mean_temp',
    assetId='projects/ml-meets-the-sun/assets/TN_mean_temp',
    region=region,
    crsTransform = [0.1, 0, -180.05, 0, -0.1, 90.05],
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()

In [38]:
"""region = karnataka.geometry()

task =ee.batch.Export.image.toDrive(
    image=result_clip['Karnataka_temp_mean'],
    description='Karnataka_temp_mean',
    folder='GEE_ML_Meets_Sun',
    region=region,
    crsTransform = era5_temp_mean.projection().transform(),
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()"""

In [37]:
karnataka_temp = ee.Image('projects/ml-meets-the-sun/assets/KA_mean_temp')
TamilNadu_temp = ee.Image('projects/ml-meets-the-sun/assets/TN_mean_temp')
Rajasthan_temp = ee.Image('projects/ml-meets-the-sun/assets/RA_mean_temp')

In [28]:
Rajasthan_temp

In [29]:
TamilNadu_temp

In [30]:
karnataka_temp

In [38]:
vis_temp = {
    'min': 250,
    'max': 320,
    'palette': [
        '000080',
        '0000d9',
        '4000ff',
        '8000ff',
        '0080ff',
        '00ffff',
        '00ff80',
        '80ff00',
        'daff00',
        'ffff00',
        'fff500',
        'ffda00',
        'ffb000',
        'ffa400',
        'ff4f00',
        'ff2500',
        'ff0a00',
        'ff00ff',
    ],
}

In [39]:
Map.addLayer(karnataka_temp, vis_temp, 'ERA5_KA_Mean_Temp', True, 0.5)
Map.addLayer(TamilNadu_temp, vis_temp, 'ERA5_TN_Mean_Temp', True, 0.5)
Map.addLayer(Rajasthan_temp, vis_temp, 'ERA5_Ra_Mean_Temp', True, 0.5)

In [40]:

Map.addLayer(Rajasthan_temp, vis_temp, 'ERA5_Ra_Mean_Temp', True, 0.5)

In [41]:
Map

Map(bottom=1464.0, center=[-23.88583769986199, 65.30273437500001], controls=(WidgetControl(options=['position'…